<a href="https://colab.research.google.com/github/takashinnosuke/study_Keras/blob/main/Boston_Housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#Boston Housingデータセットの読み込み
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = \
  boston_housing.load_data()

In [7]:
#どんなデータか見てみる
train_data.shape

(404, 13)

In [8]:
test_data.shape

(102, 13)

In [10]:
#データの正規化
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std